# Projet SDP

Agathe Gioan, Amine Larhchim, Gauthier Roy

### Chargement des modules

In [1]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

# Module relatif à Gurobi
from gurobipy import *
import json

### Chargement des données

In [2]:
# Opening JSON file
with open('toy_instance.json') as json_file:
    data = json.load(json_file)

### Définition des contraintes

In [3]:
# Implémentation Python
n_staff = len(data["staff"])
n_job = len(data["jobs"])
n_days = data["horizon"]
n_qual = len(data["qualifications"])

eps = 10**(-3)
# Instanciation du modèle
m = Model("PL modelling using matrix")

# Création d'un vecteur de 3 variables continues
v = m.addMVar(shape=(n_staff, n_job, n_days, n_qual), vtype=GRB.BINARY, name="v")

# qualification personnnel
for i in range(n_staff):
    for q in range(n_qual):
        if not(data["qualifications"][q] in data["staff"][i]["qualifications"]):
            m.addConstr(v[i,:,:,q].sum() == 0)


# Contrainte d'unicité
for i in range(n_staff):
    for t in range(n_days):
        m.addConstr(v[i,:,t,:].sum() <= 1)
        

# Contrainte de congés
for i in range(n_staff):
    for t in range(n_days):
        if t+1 in data["staff"][i]["vacations"]:
            m.addConstr(v[i,:,t,:].sum() == 0)

# Contrainte de couverture
for p in range(n_job):
    for q in range(n_qual):
        qual = data["qualifications"][q]
        if qual in data["jobs"][p]["working_days_per_qualification"].keys():
            m.addConstr(v[:,p,:,q].sum() <= data["jobs"][p]["working_days_per_qualification"][qual])
            
        
        else:
            m.addConstr(v[:,p,:,q].sum() <= 0)

# def job_real
job_real = m.addMVar(shape=(n_job), vtype=GRB.BINARY, name="job_real")
for p in range(n_job):
    n_qual_need = sum(data["jobs"][p]["working_days_per_qualification"].values())
    m.addConstr(v[:,p,:,:].sum() >= n_qual_need  - (n_qual_need+ eps)*(1-job_real[p]))
    m.addConstr(v[:,p,:,:].sum() <= n_qual_need + (n_qual_need+ eps)*(job_real[p]))

# def end_date
end_date = m.addMVar(shape=(n_job), vtype=GRB.INTEGER, lb=0, ub=n_days, name="end_date")
b_end_date = m.addMVar(shape=(n_job, n_days), vtype=GRB.BINARY, name="b_end_date")
for p in range(n_job):
    n_qual_need = sum(data["jobs"][p]["working_days_per_qualification"].values())
    m.addConstr(end_date[p] <= n_days)
    for t in range(n_days):
        m.addConstr(v[:,p,:t+1,:].sum()>= n_qual_need -eps - (n_days + eps) * (1 - b_end_date[p,t]))
        m.addConstr(v[:,p,:t+1,:].sum()<= n_qual_need - eps + (n_days + eps) * b_end_date[p,t])
        
        m.addConstr(end_date[p] <= t * b_end_date[p,t] + n_days * (1 - b_end_date[p,t]))
        m.addConstr(end_date[p] >= (t + eps) * (1 - b_end_date[p,t]))

        
# def start_date
start_date = m.addMVar(shape=(n_job), vtype=GRB.INTEGER, lb=0, ub=n_days, name="start_date")
b_start_date = m.addMVar(shape=(n_job, n_days), vtype=GRB.BINARY, name="b_start_date")
for p in range(n_job):
    n_qual_need = sum(data["jobs"][p]["working_days_per_qualification"].values())
    m.addConstr(start_date[p] <= n_days)
    for t in range(n_days):
        m.addConstr(v[:,p,:t+1,:].sum()>= eps - (n_days + eps) * (1 - b_start_date[p,t]))
        m.addConstr(v[:,p,:t+1,:].sum()<= eps + (n_days + eps) * b_start_date[p,t])
        
        m.addConstr(start_date[p] <= t * b_start_date[p,t] + n_days * (1 - b_start_date[p,t]))
        m.addConstr(start_date[p] >= (t + eps) * (1 - b_start_date[p,t]))

# def number of project per employee
project_per_employee = m.addMVar(shape=(n_staff), vtype=GRB.INTEGER, lb=0, ub=n_job, name="project_per_employee")
b_project_per_employee = m.addMVar(shape=(n_staff, n_job), vtype=GRB.BINARY, name="b_project_per_employee")
for i in range(n_staff):
    m.addConstr(project_per_employee[i] <= n_job)
    for p in range(n_job):
        m.addConstr(v[i,p,:,:].sum()>= eps - (n_days + eps) * (1 - b_project_per_employee[i,p]))
        m.addConstr(v[i,p,:,:].sum()<= eps + (n_days + eps) * b_project_per_employee[i,p])
        
    m.addConstr(project_per_employee[i] == b_project_per_employee[i,:].sum())

max_project_per_employee = m.addMVar(shape=(1), vtype=GRB.INTEGER, name="max_project_per_employee")
m.addGenConstrMax(max_project_per_employee, list(project_per_employee))

# def has_penality
has_penality = m.addMVar(shape=(n_job), vtype=GRB.BINARY, name="has_penality")
penality = m.addMVar(shape=(n_job), vtype=GRB.INTEGER, lb=0, ub=n_days*data["jobs"][0]["daily_penalty"], name="penality")
for p in range(n_job):
    due_date = data["jobs"][p]["due_date"] -1
    m.addConstr(end_date[p] >= due_date + eps - (n_days + eps) * (1 - has_penality[p]))
    m.addConstr(end_date[p] <= due_date + (n_days + eps) * (has_penality[p]))
    
    m.addConstr(penality[p] == has_penality[p] * data["jobs"][p]["daily_penalty"] * (end_date[p] - due_date))


# def duration
duration = m.addMVar(shape=(n_job), vtype=GRB.INTEGER, name="duration")
m.addConstr(duration == end_date - start_date)

max_duration = m.addMVar(shape=(1), vtype=GRB.INTEGER, name="max_duration")
m.addGenConstrMax(max_duration, list(duration))

gain = np.zeros(n_job)
for p in range(n_job):
    gain[p] = data["jobs"][p]["gain"]

# CA_per_project
CA_per_project = m.addMVar(shape=(n_job), vtype=GRB.INTEGER, name="CA_per_project")
m.addConstr(CA_per_project == job_real * (gain - penality))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-10


<MQConstr (5,) *awaiting model update*>

In [8]:
m.setObjective(-CA_per_project.sum(), GRB.MINIMIZE)
m.params.outputflag = 0
m.update()

In [43]:
def find_all_sol(m):
    i = 0
    dict_model = {}
    list_sol = []
    list_sol_for_array = []
    for max_dur in tqdm(range(n_days+1)):
        for n_proj in range(n_job+1):
            m_it = m.copy()
            m_it.addConstr(max_duration <= max_dur)
            m_it.addConstr(max_project_per_employee <= n_proj)
        
            # maj
            m_it.update()
            # Affichage en mode texte du PL

            # Résolution du PL
            m_it.optimize()
            
            if not([m_it.ObjVal, m_it.getVarByName("max_duration[0]").X, 
                                      m_it.getVarByName("max_project_per_employee[0]").X]in list_sol):
                dict_model[i] = m_it
                
                list_sol_for_array.append(np.array([m_it.ObjVal, m_it.getVarByName("max_duration[0]").X, 
                                      m_it.getVarByName("max_project_per_employee[0]").X, i]))
                
                list_sol.append([m_it.ObjVal, m_it.getVarByName("max_duration[0]").X, 
                                      m_it.getVarByName("max_project_per_employee[0]").X])
                
                i += 1
    
    return dict_model, np.array(list_sol_for_array)

In [44]:
dict_sol, array_sol = find_all_sol(m)

100%|█████████████████████████████████████████████| 6/6 [00:05<00:00,  1.10it/s]


In [45]:
array_sol

array([[  0.      ,   0.      ,   0.      ,   0.      ],
       [-20.      ,   0.      ,   1.      ,   1.      ],
       [-37.      ,   0.      ,   2.      ,   2.      ],
       [-49.      ,   0.      ,   3.      ,   3.      ],
       [-59.      ,   0.      ,   4.      ,   4.      ],
       [-30.      ,   1.      ,   1.      ,   5.      ],
       [-55.      ,   1.      ,   2.      ,   6.      ],
       [-65.      ,   1.      ,   3.      ,   7.      ],
       [-65.      ,   1.      ,   4.      ,   8.      ],
       [-42.      ,   2.      ,   1.      ,   9.      ],
       [-65.      ,   2.      ,   2.      ,  10.      ],
       [-65.      ,   2.      ,   3.      ,  11.      ],
       [-65.      ,   2.      ,   4.      ,  12.      ],
       [-42.      ,   2.      ,   1.      ,  13.      ],
       [-65.      ,   3.      ,   2.      ,  14.      ],
       [-42.000006,   2.      ,   1.      ,  15.      ],
       [-65.      ,   4.      ,   4.      ,  16.      ],
       [-65.      ,   4.      ,

In [46]:
def is_dom(sol, solutions):
    for obj in range(3):
        obj_poss = list(range(3))
        obj_poss.remove(obj)
        possible_dom = solutions[(solutions[:,obj_poss[0]]<=sol[obj_poss[0]]) & 
                                     (solutions[:,obj_poss[1]]<=sol[obj_poss[1]])]
        #print(possible_dom, sol, obj_poss)
        if sol[obj] != possible_dom[:, obj].min():
            return True
    return False
            
def keep_non_dom_sol(solutions):
    index_sol_non_dom = []
    for i in range(len(solutions)):
        dominated = is_dom(solutions[i,:], solutions)
        if not(dominated) and not(i in index_sol_non_dom):
            index_sol_non_dom.append(i)
            print('Solution', solutions[i,-1], ':',solutions[i,:3])


    print('Gurobi found', len(index_sol_non_dom), 'non dominated solutions')
    return(index_sol_non_dom)

In [31]:
indexes = keep_non_dom_sol(array_sol)

Solution 0.0 : [0. 0. 0.]
Solution 1.0 : [-20.   0.   1.]
Solution 2.0 : [-37.   0.   2.]
Solution 3.0 : [-49.   0.   3.]
Solution 4.0 : [-59.   0.   4.]
Solution 5.0 : [-59.   0.   4.]
Solution 6.0 : [0. 0. 0.]
Solution 7.0 : [-30.   1.   1.]
Solution 8.0 : [-55.   1.   2.]
Solution 9.0 : [-65.   1.   3.]
Solution 12.0 : [0. 0. 0.]
Solution 13.0 : [-42.   2.   1.]
Solution 14.0 : [-65.   2.   2.]
Solution 18.0 : [0. 0. 0.]
Solution 24.0 : [0. 0. 0.]
Solution 25.0 : [-42.000006   2.         1.      ]
Solution 30.0 : [0. 0. 0.]
Gurobi found 17 non dominated solutions


# Brouillon pour la suite

In [69]:
m.ModelSense = GRB.MINIMIZE
#m.setObjective(vec_for_sum.sum(), GRB.MAXIMIZE)

m.setObjectiveN(-CA_per_project.sum(), 0, priority=0)
m.setObjectiveN(max_duration, 1, priority=0)
m.setObjectiveN(max_project_per_employee, 2, priority=0)

In [70]:
m.params.outputflag = 0
# maj
m.update()
# Affichage en mode texte du PL
print(m.display())

# Résolution du PL
m.optimize()


None


#### Ensemble des solutions possibles

In [71]:
solutions_obj = []
# get the set of variables
x = m.getVars()

# Ensure status is optimal
assert m.Status == GRB.Status.OPTIMAL

# Query number of multiple objectives, and number of solutions
nSolutions  = m.SolCount
nObjectives = m.NumObj
print('Problem has', nObjectives, 'objectives')
print('Gurobi found', nSolutions, 'solutions')

# For each solution, print value for each objective function
solutions = []
for s in range(nSolutions):
    # Set which solution we will query from now on
    m.params.SolutionNumber = s
    
    # Print objective value of this solution in each objective
    print('Solution', s, ':', end='')
    obj_sol = []
    for o in range(nObjectives):
        # Set which objective we will query
        m.params.ObjNumber = o
        # Query the o-th objective value
        print(' ',m.ObjNVal, end='')
        obj_sol.append(m.ObjNVal)
    obj_sol.append(s)
    solutions_obj.append(obj_sol)
    print(' ')
    
    # query the full vector of the o-th solution
    solutions.append(m.getAttr('Xn',x))
    
solutions_obj = np.array(solutions_obj)

Problem has 3 objectives
Gurobi found 10 solutions
Solution 0 :  -413.0  7.0  6.0 
Solution 1 :  -411.0  6.0  6.0 
Solution 2 :  -411.0  7.0  6.0 
Solution 3 :  -411.0  8.0  6.0 
Solution 4 :  -411.0  8.0  7.0 
Solution 5 :  -411.0  10.0  6.0 
Solution 6 :  -408.0  10.0  6.0 
Solution 7 :  -402.0  10.0  6.0 
Solution 8 :  -398.0  10.0  6.0 
Solution 9 :  -398.0  11.0  6.0 


#### Ensemble des solutions non dominées

In [72]:
solutions_non_dom = []
for i in range(len(solutions_obj)):
    dominated = False
    for obj in range(3):
        obj_poss = list(range(3))
        obj_poss.remove(obj)
        possible_dom = solutions_obj[(solutions_obj[:,obj_poss[0]]==solutions_obj[i,obj_poss[0]]) & 
                                     (solutions_obj[:,obj_poss[1]]==solutions_obj[i,obj_poss[1]])]
        if solutions_obj[i,obj] != possible_dom[:, obj].min():
            dominated = True
    if not(dominated):
        solutions_non_dom.append(solutions_obj[i,:])
        print('Solution', solutions_obj[i,-1], ':',solutions_obj[i,:3])
        

print('Gurobi found', len(solutions_non_dom), 'non dominated solutions')

Solution 0.0 : [-413.    7.    6.]
Solution 1.0 : [-411.    6.    6.]
Gurobi found 2 non dominated solutions


In [73]:
solutions_obj

array([[-413.,    7.,    6.,    0.],
       [-411.,    6.,    6.,    1.],
       [-411.,    7.,    6.,    2.],
       [-411.,    8.,    6.,    3.],
       [-411.,    8.,    7.,    4.],
       [-411.,   10.,    6.,    5.],
       [-408.,   10.,    6.,    6.],
       [-402.,   10.,    6.,    7.],
       [-398.,   10.,    6.,    8.],
       [-398.,   11.,    6.,    9.]])

In [20]:
project_per_employee.X

array([3., 2., 3.])

In [21]:
max_project_per_employee.X

array([3.])

In [9]:
job_real.X

array([ 1., -0.,  1.,  1.,  1.])

In [10]:
penality.X

array([0., 9., 0., 0., 0.])

In [11]:
has_penality.X

array([0., 1., 0., 0., 0.])

In [12]:
start_date.X

array([ 1.,  5.,  2., -0.,  3.])

In [13]:
end_date.X

array([2., 5., 3., 1., 4.])

In [14]:
duration.X

array([1., 0., 1., 1., 1.])

In [15]:
max_duration.X

array([1.])

In [16]:
data

{'horizon': 5,
 'qualifications': ['A', 'B', 'C'],
 'staff': [{'name': 'Olivia',
   'qualifications': ['A', 'B', 'C'],
   'vacations': []},
  {'name': 'Liam', 'qualifications': ['A', 'B'], 'vacations': [1]},
  {'name': 'Emma', 'qualifications': ['C'], 'vacations': [2]}],
 'jobs': [{'name': 'Job1',
   'gain': 20,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'B': 1, 'C': 1}},
  {'name': 'Job2',
   'gain': 15,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'B': 2}},
  {'name': 'Job3',
   'gain': 15,
   'due_date': 4,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'C': 2}},
  {'name': 'Job4',
   'gain': 20,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'B': 2, 'C': 1}},
  {'name': 'Job5',
   'gain': 10,
   'due_date': 5,
   'daily_penalty': 3,
   'working_days_per_qualification': {'C': 2}}]}